# W02 – SQL esencial I en DuckDB (SELECT/WHERE/GROUP BY/NULLs)

## Conexión con DDIA
- **DDIA Cap. 2**: modelos de datos y lenguajes de consulta (SQL como herramienta central).
- Aquí convertimos *preguntas* en *consultas* sobre un dataset real (NASA Exoplanet Archive).

## Prerrequisitos
- Haber hecho W01A y W01B (o al menos tener Python + dependencias instaladas).
- Si no tienes `data/raw/pscomppars.csv`, el notebook lo descargará.

## Objetivos
- Crear una **vista** `raw_ps` desde un CSV.
- Usar SQL básico: `SELECT`, `WHERE`, `ORDER BY`, `LIMIT`, `GROUP BY`, `HAVING`.
- Entender `NULL`: `COUNT(*)` vs `COUNT(col)` y `COALESCE`.

## Checklist de evidencias
- [ ] Output de: `SELECT count(*) FROM raw_ps`
- [ ] 6 consultas resueltas en la sección **TU TURNO**
- [ ] 10 consultas adicionales (tarea) guardadas al final


In [2]:
import os
os.chdir("..")
os.getcwd()

'c:\\Users\\hola-\\Documents\\FisicaComputacional3-Win\\de1-fundamentos-nasa-duckdb'

In [3]:
# Setup común (cross-platform)
import sys, subprocess
from pathlib import Path
import duckdb

DB_PATH = Path("data/exoplanets.duckdb")
DB_PATH.parent.mkdir(parents=True, exist_ok=True)
con = duckdb.connect(str(DB_PATH))

def run_module(mod: str, *args: str):
    cmd = [sys.executable, "-m", mod, *args]
    print("Running:", " ".join(cmd))
    subprocess.check_call(cmd)

raw_csv = Path("data/raw/pscomppars.csv")
if not raw_csv.exists():
    # Para clase: descarga razonable. Quita --limit si quieres el subset completo.
    run_module("src.ingest.download_exoplanets", "--format", "csv", "--limit", "50000")

# DuckDB no permite parámetros preparados en DDL (ej. CREATE VIEW).
# Insertamos la ruta como literal SQL, escapando comillas simples.
def sql_quote(s: str) -> str:
    return "'" + s.replace("'", "''") + "'"

raw_csv_abs = raw_csv.resolve()
con.execute(
    f"CREATE OR REPLACE VIEW raw_ps AS SELECT * FROM read_csv_auto({sql_quote(raw_csv_abs.as_posix())})"
)
con.execute("SELECT count(*) AS n_rows FROM raw_ps").fetchall()


[(6065,)]

## DEMO

In [4]:
# DEMO 1: inspección rápida
con.sql("DESCRIBE raw_ps").show()


┌─────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│   column_name   │ column_type │  null   │   key   │ default │  extra  │
│     varchar     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ pl_name         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ hostname        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ discoverymethod │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ disc_year       │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ sy_snum         │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ sy_pnum         │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ sy_dist         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ ra              │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ dec             │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ pl_orbper       │ DOUBLE      │ YES 

In [5]:
# DEMO 2: SELECT + LIMIT (muestra pequeña)
con.sql("""
SELECT pl_name, hostname, discoverymethod, disc_year
FROM raw_ps
LIMIT 10
""").show()


┌─────────────────────────┬───────────────────────┬─────────────────┬───────────┐
│         pl_name         │       hostname        │ discoverymethod │ disc_year │
│         varchar         │        varchar        │     varchar     │   int64   │
├─────────────────────────┼───────────────────────┼─────────────────┼───────────┤
│ 11 Com b                │ 11 Com                │ Radial Velocity │      2007 │
│ 11 UMi b                │ 11 UMi                │ Radial Velocity │      2009 │
│ 14 And b                │ 14 And                │ Radial Velocity │      2008 │
│ 14 Her b                │ 14 Her                │ Radial Velocity │      2002 │
│ 16 Cyg B b              │ 16 Cyg B              │ Radial Velocity │      1996 │
│ 17 Sco b                │ 17 Sco                │ Radial Velocity │      2020 │
│ 18 Del b                │ 18 Del                │ Radial Velocity │      2008 │
│ 1RXS J160929.1-210524 b │ 1RXS J160929.1-210524 │ Imaging         │      2008 │
│ 24 Boo b      

In [6]:
# DEMO 3: WHERE + ORDER BY (evita NULL)
con.sql("""
SELECT pl_name, pl_orbper, pl_rade
FROM raw_ps
WHERE pl_orbper IS NOT NULL
ORDER BY pl_orbper ASC
LIMIT 10
""").show()


┌──────────────────┬─────────────┬─────────────┐
│     pl_name      │  pl_orbper  │   pl_rade   │
│     varchar      │   double    │   double    │
├──────────────────┼─────────────┼─────────────┤
│ PSR J1719-1438 b │ 0.090706293 │        NULL │
│ ZTF J1828+2308 b │   0.1120067 │ 11.13051784 │
│ M62H b           │ 0.132935028 │        NULL │
│ KOI-1843.03      │   0.1768913 │        0.61 │
│ K2-137 b         │    0.179719 │        0.64 │
│ KIC 10001893 b   │      0.2197 │        NULL │
│ ZTF J1230-2655 b │  0.23597766 │ 13.78704626 │
│ TOI-6255 b       │  0.23818244 │       1.079 │
│ KOI-55 b         │    0.240104 │       0.759 │
│ TOI-6324 b       │    0.279221 │       1.059 │
├──────────────────┴─────────────┴─────────────┤
│ 10 rows                            3 columns │
└──────────────────────────────────────────────┘



In [7]:
# DEMO 4: NULLs — COUNT(*) vs COUNT(col)
con.sql("""
SELECT
  COUNT(*)                    AS total_rows,
  COUNT(pl_rade)              AS non_null_radius,
  COUNT(*) - COUNT(pl_rade)   AS null_radius
FROM raw_ps
""").show()


┌────────────┬─────────────────┬─────────────┐
│ total_rows │ non_null_radius │ null_radius │
│   int64    │      int64      │    int64    │
├────────────┼─────────────────┼─────────────┤
│       6065 │            6015 │          50 │
└────────────┴─────────────────┴─────────────┘



In [8]:
# DEMO 5: GROUP BY + agregados
con.sql("""
SELECT
  discoverymethod,
  COUNT(*) AS n_planets,
  AVG(pl_rade) AS avg_radius_earth
FROM raw_ps
GROUP BY 1
ORDER BY n_planets DESC
LIMIT 10
""").show()


┌───────────────────────────────┬───────────┬────────────────────┐
│        discoverymethod        │ n_planets │  avg_radius_earth  │
│            varchar            │   int64   │       double       │
├───────────────────────────────┼───────────┼────────────────────┤
│ Transit                       │      4474 │  4.369062961830983 │
│ Radial Velocity               │      1158 │  9.811825366048186 │
│ Microlensing                  │       262 │   9.82675572519084 │
│ Imaging                       │        90 │  14.81708279069767 │
│ Transit Timing Variations     │        39 │  6.493408364210528 │
│ Eclipse Timing Variations     │        17 │ 12.893333333333338 │
│ Orbital Brightness Modulation │         9 │            9.64504 │
│ Pulsar Timing                 │         8 │  5.411333333333334 │
│ Astrometry                    │         5 │              12.52 │
│ Pulsation Timing Variations   │         2 │              12.75 │
├───────────────────────────────┴───────────┴─────────────────

In [9]:
# DEMO 6: HAVING (filtra grupos después de agrupar)
con.sql("""
SELECT
  disc_year,
  COUNT(*) AS n
FROM raw_ps
WHERE disc_year IS NOT NULL
GROUP BY 1
HAVING COUNT(*) >= 200
ORDER BY disc_year ASC
""").show()


┌───────────┬───────┐
│ disc_year │   n   │
│   int64   │ int64 │
├───────────┼───────┤
│      2014 │   869 │
│      2016 │  1496 │
│      2018 │   315 │
│      2020 │   235 │
│      2021 │   554 │
│      2022 │   369 │
│      2023 │   326 │
│      2024 │   260 │
│      2025 │   226 │
└───────────┴───────┘



### Resumen
- `GROUP BY` cambia el grano: ya no son planetas, son **grupos**.
- `COUNT(*)` cuenta filas; `COUNT(col)` ignora `NULL`.
- `HAVING` filtra **después** de agrupar (a diferencia de `WHERE`).

---

## TU TURNO (práctica guiada)
Resuelve estas consultas. Pega el output (al menos las primeras filas) en cada celda.


### 1) ¿Cuántos planetas hay por año? (top 15 años con más planetas)

In [ ]:
# TODO (1): ¿Cuántos planetas hay por año? (top 15 años con más planetas)
# Pistas: usa disc_year, filtra IS NOT NULL, GROUP BY, ORDER BY n DESC, LIMIT 15
query = """
-- TODO: escribe tu SQL aquí
"""
con.execute(query).fetchall()

### 2) Top 10 sistemas (hostname) con más planetas

In [ ]:
# TODO (2): Top 10 sistemas (hostname) con más planetas
# Pistas: GROUP BY hostname, cuenta filas, ORDER BY DESC, LIMIT 10
query = """
-- TODO: escribe tu SQL aquí
"""
con.execute(query).fetchall()

### 3) ¿Qué fracción de filas tiene `pl_bmasse` nulo?

In [ ]:
# TODO (3): ¿Qué fracción de filas tiene pl_bmasse nulo?
# Pistas: COUNT(*) total, COUNT(pl_bmasse) non_null, nulls = total - non_null
#       fracción = nulls / total (convierte a DOUBLE)
query = """
-- TODO: escribe tu SQL aquí
"""
con.execute(query).fetchall()

### 4) 10 planetas con mayor radio (pl_rade) (evita NULL)

In [ ]:
# TODO (4): 10 planetas con mayor radio (pl_rade) (evita NULL)
# Pistas: WHERE pl_rade IS NOT NULL, ORDER BY pl_rade DESC, LIMIT 10
query = """
-- TODO: escribe tu SQL aquí
"""
con.execute(query).fetchall()

### 5) Compara `COUNT(*)` vs `COUNT(disc_year)` por método

In [ ]:
# TODO (5): Compara COUNT(*) vs COUNT(disc_year) por método
# Pistas: GROUP BY discoverymethod, calcula total y non_null_year = COUNT(disc_year)
query = """
-- TODO: escribe tu SQL aquí
"""
con.execute(query).fetchall()

### 6) Resumen: por método, n_planets y mediana de periodo orbital

In [ ]:
# TODO (6): Resumen por método: n_planets y mediana del periodo orbital
# Pistas: MEDIAN(pl_orbper) (filtra NULL si aplica), GROUP BY discoverymethod
query = """
-- TODO: escribe tu SQL aquí
"""
con.execute(query).fetchall()

## Para entregar (tarea)
1) **4 consultas adicionales** (tú decides las preguntas), pero deben incluir:
   - 2 consultas de calidad (nulos, rangos, duplicados, outliers simples)
   - 2 consultas científicas: pregunta + 1–2 líneas de interpretación 

2) En `docs/decisions_log.md`: 1 decisión de hoy (con evidencia: conteos o query).

## Reflexión (bitácora)
- ¿Qué consulta te pareció más difícil y por qué?
- Si el dataset creciera 100×, ¿qué consultas crees que empeoran más?


**Entrega sugerida:** crea `docs/w02a_sql_practice.md` y pega tus 6 respuestas (1–6) + 4 consultas adicionales tuyas con resultados.
